In [1]:
from deepfunding.data import get_graph_dataframe

import polars as pl

df = get_graph_dataframe()

rows, cols = df.shape
print(f"Rows: {rows}")
print(f"Columns: {cols}")

# Get unique sources and targets
sources = df.get_column("source").unique()
targets = df.get_column("target").unique()

print("Unique sources:", len(sources))
print("Unique targets:", len(targets))

df.sample(10)

Rows: 9896
Columns: 3
Unique sources: 18
Unique targets: 4289


relation,source,target
str,str,str
"""NPM""","""https://github.com/safe-global…","""https://github.com/teambition/…"
"""NPM""","""https://github.com/web3/web3.j…","""https://github.com/gkz/type-ch…"
"""NPM""","""https://github.com/ethereum/re…","""https://github.com/postcss/pos…"
"""NPM""","""https://github.com/ethereumjs/…","""https://github.com/jfmengels/e…"
"""NPM""","""https://github.com/ethereum/re…","""https://github.com/qix-/color-…"
"""NPM""","""https://github.com/web3/web3.j…","""https://github.com/sindresorhu…"
"""NPM""","""https://github.com/ethereum/re…","""https://github.com/sindresorhu…"
"""NPM""","""https://github.com/eth-infinit…","""https://github.com/multiformat…"
"""NPM""","""https://github.com/safe-global…","""https://github.com/indutny/min…"


In [ ]:
import os
import httpx
from tqdm import tqdm

github_token = os.getenv("GITHUB_TOKEN")


def fetch_repo_info(client, repo_url):
    _, _, _, owner, repo = repo_url.rstrip("/").split("/")
    api_url = f"https://api.github.com/repos/{owner}/{repo}"

    headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {github_token}",
        "X-GitHub-Api-Version": "2022-11-28",
    }

    try:
        response = client.get(api_url, headers=headers)
        response.raise_for_status()
        return response.json()
    except httpx.HTTPError:
        return {}

# Get unique target repos
target_repos = df.get_column("target").unique().to_list()

with httpx.Client(
    transport=httpx.HTTPTransport(retries=5, verify=False),
    follow_redirects=True,
    limits=httpx.Limits(max_keepalive_connections=5, max_connections=10),
) as client:
    results = []
    for repo in tqdm(target_repos, desc="Fetching repo info"):
        result = fetch_repo_info(client, repo)
        results.append(result)

# Create DataFrame from results
repo_data = [r for r in results if r is not None]
df = pl.DataFrame(repo_data)

In [4]:
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

df.sample(10)


Number of rows: 4253
Number of columns: 87


id,node_id,name,full_name,private,owner,html_url,description,fork,url,forks_url,keys_url,collaborators_url,teams_url,hooks_url,issue_events_url,events_url,assignees_url,branches_url,tags_url,blobs_url,git_tags_url,git_refs_url,trees_url,statuses_url,languages_url,stargazers_url,contributors_url,subscribers_url,subscription_url,commits_url,git_commits_url,comments_url,issue_comment_url,contents_url,compare_url,merges_url,…,ssh_url,clone_url,svn_url,homepage,size,stargazers_count,watchers_count,language,has_issues,has_projects,has_downloads,has_wiki,has_pages,has_discussions,forks_count,mirror_url,archived,disabled,open_issues_count,license,allow_forking,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,permissions,network_count,subscribers_count,custom_properties,organization,parent,source,template_repository
i64,str,str,str,bool,struct[19],str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,i64,i64,i64,str,bool,bool,bool,bool,bool,bool,i64,null,bool,bool,i64,struct[5],bool,bool,bool,list[str],str,i64,i64,i64,str,struct[5],i64,i64,struct[7],struct[19],struct[79],struct[79],struct[80]
65322884,"""MDEwOlJlcG9zaXRvcnk2NTMyMjg4NA…","""auto-bind""","""sindresorhus/auto-bind""",false,"{""sindresorhus"",170270,""MDQ6VXNlcjE3MDI3MA=="",""https://avatars.githubusercontent.com/u/170270?v=4"","""",""https://api.github.com/users/sindresorhus"",""https://github.com/sindresorhus"",""https://api.github.com/users/sindresorhus/followers"",""https://api.github.com/users/sindresorhus/following{/other_user}"",""https://api.github.com/users/sindresorhus/gists{/gist_id}"",""https://api.github.com/users/sindresorhus/starred{/owner}{/repo}"",""https://api.github.com/users/sindresorhus/subscriptions"",""https://api.github.com/users/sindresorhus/orgs"",""https://api.github.com/users/sindresorhus/repos"",""https://api.github.com/users/sindresorhus/events{/privacy}"",""https://api.github.com/users/sindresorhus/received_events"",""User"",""public"",false}","""https://github.com/sindresorhu…","""Automatically bind methods to …",false,"""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…","""https://api.github.com/repos/s…",…,"""git@github.com:sindresorhus/au…","""https://github.com/sindresorhu…","""https://github.com/sindresorhu…",null,30,462,462,"""JavaScript""",true,false,true,false,false,false,34,null,false,false,0,"{""mit"",""MIT License"",""MIT"",""https://api.github.com/licenses/mit"",""MDc6TGljZW5zZTEz""}",true,false,false,[],"""public""",34,0,462,"""main""","{false,false,false,false,true}",34,6,null,null,null,null,null
155863221,"""MDEwOlJlcG9zaXRvcnkxNTU4NjMyMj…","""postcss-modules-local-by-defau…","""css-modules/postcss-modules-lo…",false,"{""css-modules"",12612655,""MDEyOk9yZ2FuaXphdGlvbjEyNjEyNjU1"",""https://avatars.githubusercontent.com/u/12612655?v=4"","""",""https://api.github.com/users/css-modules"",""https://github.com/css-modules"",""https://api.github.com/users/css-modules/followers"",""https://api.g

In [7]:
df.write_parquet("../data/raw/github_repository_data.parquet", compression="zstd")
